In [3]:
! pip install chembl_webresource_client

We shall work with data from the ChEMBL dataset which is  a manually curated chemical database of bioactive molecules with drug-like properties.First we will install  the ChEMBL web service package from where we can obtain data.

In [4]:
import pandas as pd
from chembl_webresource_client.new_client import new_client

new_client enables us to search and access the  whole dataset.
In this drug discovery /analysis process we should  first search for potential "Targets" for drugs i.e. , the  biomolecule with which our intended drug will interact with and inhibit it . 

In [5]:
target  = new_client.target
Cov= target.search('coronavirus')
cov_data =  pd.DataFrame.from_dict(Cov)
cov_data

In [6]:
protein = cov_data.target_chembl_id[4]
protein

The target we have chosen is the SARS coronavirus 3C-like proteinase

Next we are gonna fetch all the bioactivity data for the target protein

In [7]:
bioactivity = new_client.activity
Ba = bioactivity.filter(target_chembl_id=protein).filter(standard_type="IC50")

In [8]:
BA_data = pd.DataFrame.from_dict(Ba)
BA_data

In [9]:
BA_data.to_csv("Bioactivity of protein .csv" , index = False)

In [10]:
data = pd.read_csv("./Bioactivity of protein .csv" )
data.head(2)

### **Labeling compounds as either being active, inactive or intermediate**
The bioactivity data is in the IC50 unit. Compounds having values of less than 1000 nM will be considered to be **active** while those greater than 10,000 nM will be considered to be **inactive**. As for those values in between 1,000 and 10,000 nM will be referred to as **intermediate**. 

In [11]:
print(data[['standard_value']])

In [12]:
activity_levels = []
#setting an empty list that will get filled once we run a for loop in the column
for i in data.standard_value:
    if i >=10000:
        activity_levels.append("inactive")
    elif i <= 1000:
        activity_levels.append("active")
    else:
        activity_levels.append("intermediate")
activity_levels
        

In [13]:
type(activity_levels)

In [14]:
activity_levels = pd.Series(activity_levels)

In [15]:
type(activity_levels)

In [16]:
data.columns

In [17]:
selection = ['molecule_chembl_id', 'canonical_smiles', 'standard_value']
df = data[selection]
df.head()

In [18]:
mocast0 = pd.concat([df,activity_levels], axis=1)

In [19]:
type(mocast0)

In [20]:
! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local
! conda install -c rdkit rdkit -y
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

Christopher Lipinski, a scientist at Pfizer, came up with a set of rule-of-thumb for evaluating the druglikeness of compounds. Such druglikeness is based on the Absorption, Distribution, Metabolism and Excretion (ADME) that is also known as the pharmacokinetic profile. Lipinski analyzed all orally active FDA-approved drugs in the formulation of what is to be known as the Rule-of-Five or Lipinski's Rule.

The Lipinski's Rule stated the following:

Molecular weight < 500 Dalton


Octanol-water partition coefficient (LogP) < 5

Hydrogen bond donors < 5


Hydrogen bond acceptors < 10

In [21]:
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski

In [22]:
basedata  = np.arange(10,10)

In [23]:
basedata

In [24]:
def lipinski(smiles, verbose=False):

    moldata= []
    for elem in smiles:
        mol=Chem.MolFromSmiles(elem) 
        moldata.append(mol)
       
    baseData= np.arange(1,1)
    i=0  
    for mol in moldata:        
       
        desc_MolWt = Descriptors.MolWt(mol)
        desc_MolLogP = Descriptors.MolLogP(mol)
        desc_NumHDonors = Lipinski.NumHDonors(mol)
        desc_NumHAcceptors = Lipinski.NumHAcceptors(mol)
           
        row = np.array([desc_MolWt,
                        desc_MolLogP,
                        desc_NumHDonors,
                        desc_NumHAcceptors])   
    
        if(i==0):
            baseData=row
        else:
            baseData=np.vstack([baseData, row])
        i=i+1      
    
    columnNames=["MW","LogP","NumHDonors","NumHAcceptors"]   
    descriptors = pd.DataFrame(data=baseData,columns=columnNames)
    
    return descriptors

In [25]:
df_lipinski = lipinski(mocast0.canonical_smiles)

In [26]:
df_lipinski.head()

In [27]:
working_data = pd.concat([df_lipinski , mocast0 ] , axis = 1)
working_data.columns

In [28]:
working_data = working_data.rename(columns={0 : 'Activity_level'})
working_data.head(2)


In [29]:
def norm_value(input):
    norm = []

    for i in input['standard_value']:
        if i > 100000000:
            i = 100000000
        norm.append(i)

    input['standard_value_norm'] = norm
    x = input.drop('standard_value', 1)
        
    return x

In [30]:
wd_norm = norm_value(working_data)
wd_norm

In [31]:



def pIC50(input):
    pIC50 = []

    for i in input['standard_value_norm']:
        molar = i*(10**-9) # Converts nM to M
        pIC50.append(-np.log10(molar))

    input['pIC50'] = pIC50
    x = input.drop('standard_value_norm', 1)
        
    return x

In [32]:
final_working_data = pIC50(wd_norm)
final_working_data

In [33]:
final_working_data = final_working_data[working_data.Activity_level != 'intermediate']


In [34]:
final_working_data.Activity_level.value_counts()

In [35]:
import seaborn as sns
sns.set(style='ticks')
import matplotlib.pyplot as plt

In [36]:
plt.figure(figsize=(5.5, 5.5))

sns.countplot(x='Activity_level', data=final_working_data, edgecolor='black')

plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel('Frequency', fontsize=14, fontweight='bold')

plt.savefig('plot_bioactivity_class.pdf')

In [37]:
plt.figure(figsize=(5.5, 5.5))

Plot = sns.scatterplot(x='MW', y='LogP', data = final_working_data, hue='Activity_level', size='pIC50', edgecolor='black', alpha=0.7)
Plot.axvline(500)
Plot.axhline(5)
plt.xlabel('MW', fontsize=14, fontweight='bold')
plt.ylabel('LogP', fontsize=14, fontweight='bold')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0)
plt.savefig('plot_MW_vs_LogP.pdf')

In [48]:
plt.figure(figsize=(5.5, 5.5))

sns.boxplot(x = 'Activity_level', y = 'pIC50', data = final_working_data)

plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel('pIC50 value', fontsize=14, fontweight='bold')



In [46]:
def mannwhitney(descriptor, verbose=False):
    from numpy.random import seed
    from numpy.random import randn
    from scipy.stats import mannwhitneyu
    # seed the random number generator
    seed(1)
    # actives and inactives
    selection = [descriptor,'Activity_level']

    df = final_working_data[selection]
    active = df[df.Activity_level == 'active']
    active = active[descriptor]

    selection = [descriptor, 'Activity_level']
    df = final_working_data[selection]
    inactive = df[df.Activity_level == 'inactive']
    inactive = inactive[descriptor]
    # compare samples
    stat, p = mannwhitneyu(active, inactive)
    #print('Statistics=%.3f, p=%.3f' % (stat, p))
    # interpret
    alpha = 0.05
    if p > alpha:
        interpretation = 'Same distribution (fail to reject H0)'
    else:
        interpretation = 'Different distribution (reject H0)'
    results = pd.DataFrame({'Descriptor':descriptor,
                          'Statistics':stat,
                          'p':p,
                          'alpha':alpha,
                          'Interpretation':interpretation}, index=[0])
    return results

In [47]:
mannwhitney('pIC50')

In [49]:
plt.figure(figsize=(5.5, 5.5))

sns.boxplot(x = 'Activity_level', y = 'MW', data = final_working_data)

plt.xlabel('Activity_level', fontsize=14, fontweight='bold')
plt.ylabel('MW', fontsize=14, fontweight='bold')



In [50]:
mannwhitney('MW')

In [51]:
plt.figure(figsize=(5.5, 5.5))

sns.boxplot(x = 'Activity_level', y = 'LogP', data = final_working_data)

plt.xlabel('Activity_level', fontsize=14, fontweight='bold')
plt.ylabel('LogP', fontsize=14, fontweight='bold')


In [52]:
mannwhitney('LogP')

In [53]:
plt.figure(figsize=(5.5, 5.5))

sns.boxplot(x = 'Activity_level', y = 'NumHDonors', data = final_working_data)

plt.xlabel('Activity_level', fontsize=14, fontweight='bold')
plt.ylabel('NumHDonors', fontsize=14, fontweight='bold')



In [55]:
mannwhitney('NumHDonors')

In [57]:
plt.figure(figsize=(5.5, 5.5))

sns.boxplot(x = 'Activity_level', y = 'NumHAcceptors', data = final_working_data)

plt.xlabel('Activity_level', fontsize=14, fontweight='bold')
plt.ylabel('NumHAcceptors', fontsize=14, fontweight='bold')

In [58]:
mannwhitney('NumHAcceptors')